## R2 To R2 Attractor, Position from a Point

In [1]:
using PBDS, StaticArrays, LinearAlgebra, BenchmarkTools

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-4224103349786716603\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-527399712171607831\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-4224103349786716603\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

## Setup

In [2]:
PBDS.metric_chart(xn, task::Attractor{<:PositionAroundPoint{R2,R2,S}}, CN::Chart{1,R2}) where S = default_metric(xn, task, CN)
PBDS.potential_chart(xn, task::Attractor{<:PositionAroundPoint{R2,R2,S}}, CN::Chart{1,R2}) where S = sum(xn.^2)
PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:PositionAroundPoint{R2,R2,S}}, CN::Chart{1,R2}) where S = -3*vn
PBDS.weight_metric_chart(xn, vn, task::Attractor{<:PositionAroundPoint{R2,R2,S}}, CN::Chart{1,R2}) where S = default_weight_metric(xn, vn, task, CN)

In [3]:
xm_goal = SA[0., 0.]
M, N = R2, R2
task = Attractor(PositionAroundPoint{M,N}(xm_goal))
CM, CN = Chart{1,M}(), Chart{1,N}()

(Chart{1,ℝ{2}}(), Chart{1,ℝ{2}}())

## Point Acceleration

In [4]:
# Initial state
xm = SA[1., 2.]
vm = SA[1., 1.]
σxddot, = single_task_acceleration(xm, vm, task, CM, CN)

([-5.0, -7.0], Chart{1,ℝ{2}}())

## Single Trajectory

In [5]:
Time = 10
dt = 0.05

robot_coord_rep = ChartRep()
traj = propagate_task(xm, vm, task, CM, CN, Time, dt, robot_coord_rep)
traj.xm[end]

2-element SArray{Tuple{2},Float64,1,2} with indices SOneTo(2):
 0.0001431784232347343
 0.00023863146471392462

## Multiple Trajectories

In [6]:
using Makie, Observables, ProgressMeter, VectorizedRoutines

In [7]:
Time = 5
dt = 0.02
xlim, Δx = 1., 1.
vlim, Δv = 1., 2.

xm0 = Array{Any}(undef, 2)
vm0 = Array{Any}(undef, 2)

xm0[1], xm0[2] = Matlab.meshgrid(-xlim:Δx:xlim, -xlim:Δx:xlim)
vm0[1], vm0[2] = Matlab.meshgrid(-vlim:Δv:vlim, -vlim:Δv:vlim)

for a in (xm0, vm0), i in 1:2
    a[i] = reshape(a[i], length(a[i]))
end

nx = length(xm0[1])
nv = length(vm0[1])
ix, iv = Matlab.meshgrid(1:nx, 1:nv)
ix = reshape(ix, length(ix))
iv = reshape(iv, length(iv))

ntraj = nx*nv
trajs = Array{Any}(undef, ntraj)
xm0 = [xm0[1]'; xm0[2]']
vm0 = [vm0[1]'; vm0[2]']
p = Progress(ntraj)

Threads.@threads for i in 1:ntraj
    trajs[i] = propagate_task(SA[xm0[:,ix[i]]...], SA[vm0[:,iv[i]]...], task, CM, CN, Time, dt, robot_coord_rep)
    next!(p)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:06


In [8]:
fxx(i) = [trajs[j].xm[i][1] for j = 1:ntraj]
fxy(i) = [trajs[j].xm[i][2] for j = 1:ntraj]
iobs = Observable(1)

ax_size, plot_size = 2, 500
limits = FRect((-ax_size, -ax_size), (2*ax_size, 2*ax_size))
scene = Scene(resolution = (plot_size, plot_size))
Makie.scatter!(scene, lift(i -> fxx(i), iobs), lift(i -> fxy(i), iobs), markersize = ax_size/20, color = :blue, limits = limits)
Makie.scatter!(scene, xm_goal', markersize = ax_size/20, color = :darkgreen)
for i = 1:ntraj
    Makie.lines!(scene, getindex.(trajs[i].xm,1), getindex.(trajs[i].xm,2), color = :purple)
end

Makie.xlabel!(scene, "x")
Makie.ylabel!(scene, "y")
axis = scene[Axis]
axis.showaxis = false
display(scene)

GLMakie.Screen(...)

In [9]:
function record_scene(scene, filename, iobs, N)
    p = Progress(N)
    record(scene, filename, 1:N) do i
        iobs[] = i
        next!(p)
    end
    display("text/html", html_video(filename))
end

filename = "R2ToR2Attractor.mp4"
record_scene(scene, filename, iobs, length(trajs[1].xm))

Progress: 100%|█████████████████████████████████████████| Time: 0:00:12
